In [19]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import interpolate

In [ ]:
#Relajación

def initialize_grid(N, M, a, b, v0):
  """
    Inicializa la rejilla con condiciones de contorno.

    Parámetros:
    - N, M: Número de puntos en las direcciones x e y, respectivamente.
    - a, b: Ancho y altura del dominio rectangular.
    - v0: Valor de la condición de contorno en y=0 e y=b.

    Devoluciones:
    - V: La rejilla inicializada.

  """
  V = np.zeros((N, M))
    # Aplicar condiciones de contorno
  V[0,:] = -v0  # y=0
  V[-1,:] = v0  # y=b
  return V

def relax_potential(V, N, M):
    """
    Realiza un paso de relajación único en la rejilla de potencial.

    Parámetros:
    - V: La rejilla de valores de potencial
    - N, M: Número de puntos en las direcciones x e y, respectivamente.

    Devoluciones:
    - V_nuevo: La rejilla actualizada después de un paso de relajación.
    """
    V_new = V.copy()
    for i in range(1, N-1):
        for j in range(1, M-1):
            # Actualizar el potencial basado en el promedio de los vecinos
            V_new[i, j] = 0.25 * (V[i+1, j] + V[i-1, j] + V[i, j+1] + V[i, j-1])
    return V_new

def solve_laplace(N, M, dx, a, b, v0, tol=1e-5):
    """
    Resuelve la ecuación de Laplace usando el método de relajación.

    Parámetros:
    - N, M: Número de puntos en las direcciones x e y, respectivamente.
    - dx: Tamaño del paso en ambas direcciones x e y.
    - a, b: Ancho y altura del dominio rectangular.
    - v0: Valor de la condición de contorno en y=0 e y=b.
    - tol: Tolerancia para el cambio de potencial para determinar la convergencia.

    Devoluciones:
    - V: La rejilla convergida de valores de potencial.
    """
    # Inicializar la rejilla con condiciones de contorno
    V = initialize_grid(N, M, a, b, v0)

    # Implementat relajación hasta que el cambio esté por debajo de la tolerancia
    while True:
        V_new = relax_potential(V, N, M)
        delta_V = np.max(np.abs(V_new - V))
        V = V_new
        if delta_V < tol:
            break

    return V

def main():
    """
    Función principal para ejecutar el solucionador de Laplace con entradas del usuario.
    """
    # Entradas del usuario
    N = int(input("Enter the number of points in the x direction (N): "))
    M = int(input("Enter the number of points in the y direction (M): "))
    dx = float(input("Enter the step size (dx=dy): "))
    a = float(input("Enter the width of the domain (a): "))
    b = float(input("Enter the height of the domain (b): "))
    v0 = float(input("Enter the boundary condition value (v0): "))

    # Solucionar la ecuacipon de Laplace

    V = solve_laplace(N, M, dx, a, b, v0)

    # Mostrar los resultados
    print("The potential distribution has been calculated.")

    # Opcional: Trazar los resultados si matplotlib está disponible
    try:
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D

        plt.imshow(V, cmap='hot', interpolation='nearest')
        plt.colorbar(label='Potential (V)')
        plt.title('2D 1Potential Distribution of Laplace Equation')
        plt.xlabel('x index')
        plt.ylabel('y index')
        plt.show()


        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        X, Y = np.meshgrid(np.arange(N), np.arange(M))
        ax.plot_surface(X, Y, V, cmap='hot')
        ax.set_xlabel('X index')
        ax.set_ylabel('Y index')
        ax.set_zlabel('Potential (V)')
        ax.set_title('3D Potential Distribution of Laplace Equation')
        plt.show()


    except ImportError:
        print("Matplotlib is not installed. Only numerical results are available.")
        print(V)

if __name__ == "__main__":
    main()


In [ ]:
# Separación de variables

# Constantes
V0 = 1  # Constante del potencial inicial
a = 50  # Ancho de la regipon rectangular
b = 50  # Altura de la región rectangular
n_terms = 100  # Número de términos de la sumatoria

def V(x, y, V0, a, b, n_terms):
    sum_series = 0
    for n in range(1, n_terms + 1):
        term = ((2 * V0 * np.sinh(n * np.pi * y / a) - V0 * np.sinh(n * np.pi * (b - y) / a)) / np.sinh(n * np.pi * b / a)) * np.sin(n * np.pi * x / a) / n
        sum_series += term
    return (4 / np.pi) * sum_series

x = np.arange(0, a + 10, 10)
y = np.arange(0, b + 10, 10)
X, Y = np.meshgrid(x, y)
V_xy = np.vectorize(V)(X, Y, V0, a, b, n_terms)

fig, ax = plt.subplots(figsize=(7, 6))
contour = ax.contourf(X, Y, V_xy, levels=50, cmap='hot')
fig.colorbar(contour, ax=ax, label='Potential V(x, y)')
ax.set_title('Potential V(x, y)')
ax.set_xlabel('X index')
ax.set_ylabel('Y index')
plt.show()


In [ ]:
#Error entre los métodos

# Constantes
V0 = 1  # Constante del potencial inicial
a = 50  # Ancho de la regipon rectangular
b = 50  # Altura de la región rectangular
n_terms = 100  # Número de términos de la sumatoria


def V_analytical(x, y, V0, a, b, n_terms):
    sum_series = 0
    for n in range(1, n_terms + 1):
        term = ((2 * V0 * np.sinh(n * np.pi * y / a) - V0 * np.sinh(n * np.pi * (b - y) / a)) / np.sinh(n * np.pi * b / a)) * np.sin(n * np.pi * x / a) / n
        sum_series += term
    return (4 / np.pi) * sum_series

def initialize_grid(N, M, a, b, v0):
    """
    Inicializa la cuadrícula con condiciones de contorno.

    Parámetros:
    - N, M: Número de puntos en las direcciones x e y, respectivamente.
    - a, b: Ancho y alto del dominio rectangular.
    - v0: Valor de la condición de contorno en y=0 y y=b.

    Retorna:
    - V: La cuadrícula inicializada.
     """
    V = np.zeros((N, M))
    # Apply boundary conditions
    V[-1,:] = -v0  # y=0
    V[0,:] = v0  # y=b
    return V

def relax_potential(V, N, M):
    """
    Realiza un paso de relajación en la cuadrícula de potenciales.

    Parámetros:
    - V: La cuadrícula de valores potenciales.
    - N, M: Número de puntos en las direcciones x e y, respectivamente.

    Retorna:
    - V_new: La cuadrícula actualizada después de un paso de relajación.
    """
    V_new = V.copy()
    for i in range(1, N-1):
        for j in range(1, M-1):
            # Update the potential based on the average of the neighbors
            V_new[i, j] = 0.25 * (V[i+1, j] + V[i-1, j] + V[i, j+1] + V[i, j-1])
    return V_new

def solve_laplace(N, M, dx, a, b, v0, tol=1e-5):
    """
    Resuelve la ecuación de Laplace usando el método de relajación.

    Parámetros:
    - N, M: Número de puntos en las direcciones x e y, respectivamente.
    - dx: Tamaño del paso en las direcciones x e y.
    - a, b: Ancho y alto del dominio rectangular.
    - v0: Valor de la condición de contorno en y=0 y y=b.
    - tol: Tolerancia para el cambio potencial para determinar la convergencia.

    Retorna:
    - V: La cuadrícula convergida de valores potenciales.
    """
    # Iniciar rejilla con las condiciones de frintera
    V = initialize_grid(N, M, a, b, v0)

    # Relajación hasta estar debajo de la tolerancia
    while True:
        V_new = relax_potential(V, N, M)
        delta_V = np.max(np.abs(V_new - V))
        V = V_new
        if delta_V < tol:
            break

    return V

# Solución analítica
x = np.arange(0, a + 10, 10)
y = np.arange(0, b + 10, 10)
X, Y = np.meshgrid(x, y)
V_analytical_xy = V_analytical(X, Y, V0, a, b, n_terms)

# Solución numérica
N = len(x)
M = len(y)
dx = 10
V_numerical = solve_laplace(N, M, dx, a, b, V0)

# Calculo del error
error = np.abs(V_analytical_xy - V_numerical)

# Interpolar para suavizar la gráfica
f_interp = interpolate.interp2d(x, y, error, kind='cubic')
x_interp = np.linspace(0, a, 100)
y_interp = np.linspace(0, b, 100)
error_interp = f_interp(x_interp, y_interp)

#Plot
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)

#Grpafica 3D
X_3d_interp, Y_3d_interp = np.meshgrid(x_interp, y_interp)
ax1.plot_surface(X_3d_interp, Y_3d_interp, error_interp, cmap='hot')
ax1.set_title('Error between Numerical and Analytical solutions')
ax1.set_xlabel('X index')
ax1.set_ylabel('Y index')
ax1.set_zlabel('Error')

#Contorno (2D)
contour = ax2.contourf(X, Y, error, levels=50, cmap='hot')
fig.colorbar(contour, ax=ax2, label='Error')
ax2.set_title('Error Contour Plot')
ax2.set_xlabel('x')
ax2.set_ylabel('y')

plt.tight_layout()
plt.show()
